# Impyte Documentation
This is a first practicle attempt to clarify the usage of `impyte`. It's a collection of easy applicable and reproducible examples that you could use to simplyify your data processing workflow.

## Importing and generating sample data
In order to show some of the features of the library, we'll be using Kaggle's HR data that can be found [here](https://www.kaggle.com/ludobenistant/hr-analytics).

In [19]:
# import library and data set
import impyte
import pandas as pd
from tools.data_prep import remove_random

In [20]:
data = pd.read_csv('data/hr_test.csv')

In [21]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Add random missing values
In order to show some of the pattern visualization and imputation methods, we need incomplete data sets. To achieve this, we'll be using a helper function that deletes values at random. 25 % of all values from each category will be deleted.

In [22]:
data = remove_random(data, .25)

In [23]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.58,5.0,182.0,2.0,0.0,0.0,0.0,technical,medium
1,NaN,0.48,2.0,152.0,3.0,0.0,1.0,0.0,IT,low
2,NaN,0.89,3.0,242.0,3.0,0.0,0.0,0.0,product_mng,medium
3,NaN,0.87,4.0,144.0,3.0,0.0,0.0,0.0,support,low
4,NaN,0.53,6.0,173.0,4.0,0.0,0.0,0.0,sales,low


In [24]:
data.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000
mean,0.612574,0.716356,3.802530,201.027009,3.500034,0.144068,0.238154,0.021333
std,0.248544,0.171270,1.231752,49.982793,1.462053,0.351171,0.425968,0.144498
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [25]:
len(data)

14999

In [75]:
reload(impyte)
imputer = impyte.Imputer() # instantiate impyte class
pattern_log = impyte.Pattern()

## Testing rudimentary features
Below is a first trial run of some of the features implemented by impyter and its helper classes.

## `NanChecker`
Functionality testing of `NanChecker` class.

In [89]:
nan_checker = impyte.NanChecker()

#### `NanChecker.is_nan( )`
The NaN detection feature that also enables NaN-value ingestion as parameter values.

In [77]:
# [True, False, False]
print nan_checker.is_nan(["", 'None', 'NaN'])

# [True, True, True]
print nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

[True, False, False]
[True, True, True]


### Timing some of the functions
Below is a piece of information on efficiency of the functions. This doesn't reflect complexity of the functions, but it will show an runtime estimate for your local machine.

In [78]:
%timeit nan_checker.is_nan(["", 'None', 'NaN'])
%timeit nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

100000 loops, best of 3: 5.34 µs per loop
100000 loops, best of 3: 4.95 µs per loop


## `Pattern`
Functionality testing of `Pattern` class. The `Pattern` class stores different patterns and data summaries regarding NaN values. 

#### `Pattern._get_discrete_and_continuous(tmpdata)`
Returns the column names of discrete and continuous variables. Column names are stored in lists for easy selection. Those lists are stored in one dictionary object. All continuous column names can be accessed through `['continuous']` all discrete variables through `['discrete']`.

In [90]:
pattern_log._get_discrete_and_continuous(data)

{'continuous': ['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company',
  'Work_accident',
  'left',
  'promotion_last_5years'],
 'discrete': ['sales', 'salary']}

#### `Pattern._compute_pattern(data, nan_values="", verbose=False)`
Function that checks for missing values and prints out a quick table of a summary of missing values. Includes pattern overview and counts of missing values by column. Computes pattern and returns its pattern structure plus the count of data points for each of these patterns. To get a nice digestible table add `["table"]` selector to output.

In [120]:
pattern_dict = pattern_log._compute_pattern(data)
indices, table = pattern_dict["indices"], pattern_dict["table"]

In [92]:
table

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
10,1,1,1,1,1,1,1,1,1,1,11259
8,1,1,1,1,1,1,1,1,NaN,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374
7,1,1,1,1,1,1,1,NaN,1,1,374
4,1,1,1,1,NaN,1,1,1,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374


In [93]:
indices[0][:10] # first 10 indices of pattern 0

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

#### `Pattern.get_pattern()`
Returns NaN-patterns based on primary computation or initiates new computation of NaN-patterns. Uses `_compute_patterns()` to visualize patterns if not yet computed. Otherwise returns stored information regarding patterns.

In [94]:
pattern_log.get_pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
10,1,1,1,1,1,1,1,1,1,1,11259
8,1,1,1,1,1,1,1,1,NaN,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374
7,1,1,1,1,1,1,1,NaN,1,1,374
4,1,1,1,1,NaN,1,1,1,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374


#### `Pattern.row_nan_pattern( )`
This is a core piece to determining which patterns are available. Other methods work with the output of `row_nan_pattern()` to determine unique pattern structures and count these before turning them into readable tables.

In [95]:
# (1, 1, 'NaN', 'NaN')
print pattern_log.row_nan_pattern(['Value 1', 'Value 2', '', None])

(1, 1, 'NaN', 'NaN')


#### `Pattern.get_pattern_indices(pattern_no)`

In [98]:
pattern_log.get_pattern_indices(0)[:10] # get first 10 indices of pattern 0

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

#### `Pattern.print_pattern(data)`
Counts individual NaN patterns and returns them in a dictionary.

In [119]:
print_pattern_dict = pattern_log.print_pattern(data)

In [112]:
print_pattern_dict

Counter({(1, 1, 1, 1, 1, 1, 1, 1, 1, 1): 11259,
         (1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN'): 374,
         (1, 1, 1, 1, 1, 1, 1, 1, 'NaN', 1): 374,
         (1, 1, 1, 1, 1, 1, 1, 'NaN', 1, 1): 374,
         (1, 1, 1, 1, 1, 1, 'NaN', 1, 1, 1): 374,
         (1, 1, 1, 1, 1, 'NaN', 1, 1, 1, 1): 374,
         (1, 1, 1, 1, 'NaN', 1, 1, 1, 1, 1): 374,
         (1, 1, 1, 'NaN', 1, 1, 1, 1, 1, 1): 374,
         (1, 1, 'NaN', 1, 1, 1, 1, 1, 1, 1): 374,
         (1, 'NaN', 1, 1, 1, 1, 1, 1, 1, 1): 374,
         ('NaN', 1, 1, 1, 1, 1, 1, 1, 1, 1): 374})

#### `Pattern.get_continuous()`
Returns list with names of all continuous variables.

In [115]:
pattern_log.get_continuous()

['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years']

#### `Pattern.get_discrete()`
Returns list with names of all discrete variables.

In [116]:
pattern_log.get_discrete()

['sales', 'salary']

## `Imputer`

### Load data into imputer

In [35]:
reload(impyte)
imputer.load_data(data)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.58,5.0,182.0,2.0,0.0,0.0,0.0,technical,medium
1,NaN,0.48,2.0,152.0,3.0,0.0,1.0,0.0,IT,low
2,NaN,0.89,3.0,242.0,3.0,0.0,0.0,0.0,product_mng,medium
3,NaN,0.87,4.0,144.0,3.0,0.0,0.0,0.0,support,low
4,NaN,0.53,6.0,173.0,4.0,0.0,0.0,0.0,sales,low
5,NaN,0.68,4.0,159.0,3.0,0.0,0.0,0.0,sales,low
6,NaN,0.77,4.0,180.0,3.0,0.0,0.0,0.0,sales,medium
7,NaN,0.45,2.0,140.0,3.0,0.0,1.0,0.0,sales,medium
8,NaN,0.72,2.0,180.0,2.0,1.0,0.0,0.0,technical,low
9,NaN,1.00,3.0,255.0,3.0,0.0,0.0,0.0,sales,medium


In [36]:
%timeit imputer.pattern()

I'm there !
I'm here
I'm here
I'm here
The slowest run took 85856.49 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 22.9 µs per loop


In [37]:
%timeit imputer.get_pattern(4)

The slowest run took 4.05 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 701 µs per loop


In [38]:
reload(impyte)
imputer = impyte.Imputer(data) # instantiate impyte class
nan_checker = impyte.NanChecker()
pattern_log = impyte.Pattern()
imputer.pattern()

I'm there !


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
10,1,1,1,1,1,1,1,1,1,1,11259
8,1,1,1,1,1,1,1,1,NaN,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374
7,1,1,1,1,1,1,1,NaN,1,1,374
4,1,1,1,1,NaN,1,1,1,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374


In [39]:
_ = imputer.drop_pattern(4, inplace=True)

In [40]:
imputer.pattern()

I'm here


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
10,1,1,1,1,1,1,1,1,1,1,11259
8,1,1,1,1,1,1,1,1,NaN,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374
7,1,1,1,1,1,1,1,NaN,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374
6,1,1,1,1,1,1,NaN,1,1,1,374


In [41]:
_ = imputer.load_data(data)

In [42]:
imputer.pattern()

I'm there !


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
10,1,1,1,1,1,1,1,1,1,1,11259
8,1,1,1,1,1,1,1,1,NaN,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374
7,1,1,1,1,1,1,1,NaN,1,1,374
4,1,1,1,1,NaN,1,1,1,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374


In [43]:
_ = imputer.drop_pattern(4)

In [44]:
imputer.pattern()

I'm here


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
10,1,1,1,1,1,1,1,1,1,1,11259
8,1,1,1,1,1,1,1,1,NaN,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374
7,1,1,1,1,1,1,1,NaN,1,1,374
4,1,1,1,1,NaN,1,1,1,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374


In [45]:
imputer.drop_pattern(7, inplace=True)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.58,5.0,182.0,2.0,0.0,0.0,0.0,technical,medium
1,NaN,0.48,2.0,152.0,3.0,0.0,1.0,0.0,IT,low
2,NaN,0.89,3.0,242.0,3.0,0.0,0.0,0.0,product_mng,medium
3,NaN,0.87,4.0,144.0,3.0,0.0,0.0,0.0,support,low
4,NaN,0.53,6.0,173.0,4.0,0.0,0.0,0.0,sales,low
5,NaN,0.68,4.0,159.0,3.0,0.0,0.0,0.0,sales,low
6,NaN,0.77,4.0,180.0,3.0,0.0,0.0,0.0,sales,medium
7,NaN,0.45,2.0,140.0,3.0,0.0,1.0,0.0,sales,medium
8,NaN,0.72,2.0,180.0,2.0,1.0,0.0,0.0,technical,low
9,NaN,1.00,3.0,255.0,3.0,0.0,0.0,0.0,sales,medium


In [46]:
imputer.pattern()

I'm here


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
10,1,1,1,1,1,1,1,1,1,1,11259
8,1,1,1,1,1,1,1,1,NaN,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374
4,1,1,1,1,NaN,1,1,1,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374
6,1,1,1,1,1,1,NaN,1,1,1,374


In [47]:
imputer.impute()

(['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company',
  'Work_accident',
  'left',
  'promotion_last_5years'],
 ['sales', 'salary'])

In [45]:
data.tail()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
14994,0.52,0.94,3.0,263.0,3.0,0.0,0.0,0.0,product_mng,medium
14995,0.43,0.44,5.0,213.0,3.0,0.0,0.0,0.0,sales,low
14996,0.45,0.49,2.0,134.0,3.0,0.0,1.0,0.0,RandD,medium
14997,0.10,0.90,7.0,281.0,4.0,0.0,1.0,0.0,sales,low
14998,0.54,0.67,2.0,129.0,3.0,1.0,0.0,0.0,sales,low


In [35]:
nnna = {}

In [39]:
if not nnna:
    print "yes"

yes
